<a href="https://colab.research.google.com/github/pedrobcavalcante/Backtest-trading-strategies/blob/main/Gerador_de_Par%C3%A2metros_para_estrat%C3%A9gias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [1]:
import backtesting 
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import requests
import json
import pandas as pd
import datetime as dt
backtesting.set_bokeh_output(notebook=False)


C:\Users\pedro\anaconda3\lib\site-packages\backtesting\_plotting.py:47: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
from backtesting.test import SMA


In [3]:
def fixData(data):
    columns = ['open_time', 'Open', 'High', 'Low', 'Close', 'Volume',
               'close_time', 'qav', 'num_trades',
               'taker_base_vol', 'taker_quote_vol', 'ignore']
    df = pd.DataFrame(data, columns=columns)
    df.index = [dt.datetime.fromtimestamp(x/1000.0) for x in df.close_time]
    df.sort_values(["open_time"], ascending=False)
    df = df.astype('double')
    df['Volume'] = df['Volume'].astype('int')
    df = df.drop(['qav', 'num_trades', 'taker_base_vol',
                 'taker_quote_vol', 'ignore'], axis=1)
    return df


In [4]:
def get_bars(symbol, interval='1h'):
    root_url = 'https://api.binance.com/api/v1/klines'
    filename = "data/"+symbol+"_"+interval+".csv"
    url = root_url + '?symbol=' + symbol + '&interval=' + interval+"&limit=1001"
    print(url)
    data = json.loads(requests.get(url).text)
    df = fixData(data)
#    df.to_csv(filename, sep=';')
    return df


In [5]:
class SmaCross(Strategy):
    n1 = 0
    n2 = 0

    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.High, self.n1)
        self.sma2 = self.I(SMA, self.data.Low, self.n2)

    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.data.Close, self.sma1):
            # self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.data.Close):
            self.position.close()
            # self.sell()

    def setInitialParams(self, valor1, valor2):
        self.n1 = valor1
        self.n2 = valor2


# Nova seção


In [6]:

data = get_bars("BTCUSDT", "1d")
SmaCross.setInitialParams(SmaCross,5,5)
bt = Backtest(data, SmaCross,cash=1000000,exclusive_orders=False)
output = bt.run()
print(output)
bt.plot()

https://api.binance.com/api/v1/klines?symbol=BTCUSDT&interval=1w&limit=1001


In [7]:
from datetime import datetime
import backtrader as bt

# Create a subclass of SignaStrategy to define the indicators and signals

class SmaCross(bt.SignalStrategy):
    # list of parameters which are configurable for the strategy
    params = dict(
        pfast=10,  # period for the fast moving average
        pslow=30   # period for the slow moving average
    )

    def __init__(self):
        sma1 = bt.ind.SMA(period=self.p.pfast)  # fast moving average
        sma2 = bt.ind.SMA(period=self.p.pslow)  # slow moving average
        crossover = bt.ind.CrossOver(sma1, sma2)  # crossover signal
        self.signal_add(bt.SIGNAL_LONG, crossover)  # use it as LONG signal


cerebro = bt.Cerebro()  # create a "Cerebro" engine instance

# Create a data feed


In [8]:
getData =  get_bars("BTCUSDT", "1d")
getData
data = bt.feeds.PandasData(dataname =getData)
getData

https://api.binance.com/api/v1/klines?symbol=BTCUSDT&interval=1d&limit=1001


,open_time,Open,High,Low,Close,Volume,close_time
2019-02-05 20:59:59.999,1.549325e+12,3463.22,3478.97,3448.43,3471.59,25264,1.549411e+12
2019-02-06 20:59:59.999,1.549411e+12,3471.57,3482.72,3380.00,3405.37,35310,1.549498e+12
2019-02-07 20:59:59.999,1.549498e+12,3407.00,3426.45,3390.00,3398.40,18665,1.549584e+12
2019-02-08 20:59:59.999,1.549584e+12,3398.40,3733.58,3373.10,3659.04,47968,1.549670e+12
2019-02-09 20:59:59.999,1.549670e+12,3660.27,3680.02,3625.13,3665.18,24759,1.549757e+12
...,...,...,...,...,...,...,...
2021-10-27 20:59:59.999,1.635293e+12,60328.81,61496.00,58000.00,58413.44,62124,1.635379e+12
2021-10-28 20:59:59.999,1.635379e+12,58413.44,62499.00,57820.00,60575.89,61056,1.635466e+12
2021-10-29 20:59:59.999,1.635466e+12,60575.90,62980.00,60174.81,62253.71,43973,1.635552e+12
2021-10-30 20:59:59.999,1.635552e+12,62253.70,62359.25,60673.00,61859.19,31478,1.635638e+12


In [9]:

cerebro.adddata(data)  # Add the data feed

cerebro.addstrategy(SmaCross)  # Add the trading strategy
cerebro.run()  # run it all
cerebro.plot()  # and plot it with a single command

ImportError: cannot import name 'warnings' from 'matplotlib.dates' (C:\Users\pedro\anaconda3\lib\site-packages\matplotlib\dates.py)